# Segmente
Cum influențează segmentul tratamentul și rata de recidivă

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact, kruskal
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter

# Load the data
df = pd.read_csv('anonim-clean.csv')

# Preprocessing
df['treatment_type'] = np.where(df['tratament'].str.contains('chirurgical', case=False, na=False), 'Surgical', 'Conservative')
df['recidivă'] = df['recidivă'].fillna(False).astype(bool)

# Extract unique segments
all_segments = df['segment'].str.split(',', expand=True).stack().str.strip().unique()
for segment in all_segments:
    df[f'segment_{segment}'] = df['segment'].str.contains(segment, case=False, na=False)

segments = [f'segment_{seg}' for seg in all_segments]

def perform_statistical_test(data, var1, var2):
    contingency_table = pd.crosstab(data[var1], data[var2])
    if (contingency_table.values < 5).any():
        _, p_value = fisher_exact(contingency_table)
        test_type = "Fisher's exact test"
    else:
        _, p_value, _, _ = chi2_contingency(contingency_table)
        test_type = "Chi-square test"
    return test_type, p_value

# Analysis for treatment decision
treatment_results = []
for segment in segments:
    test_type, p_value = perform_statistical_test(df, segment, 'treatment_type')
    treatment_results.append({
        'Segment': segment.replace('segment_', ''),
        'Test Type': test_type,
        'p-value': p_value,
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })

treatment_results_df = pd.DataFrame(treatment_results)
print("Segment vs Treatment Decision:")
print(treatment_results_df.to_string(index=False))
treatment_results_df.to_csv('segment_vs_treatment.csv', index=False)

# Analysis for recurrence rate
recurrence_results = []
for segment in segments:
    test_type, p_value = perform_statistical_test(df, segment, 'recidivă')
    recurrence_results.append({
        'Segment': segment.replace('segment_', ''),
        'Test Type': test_type,
        'p-value': p_value,
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })

recurrence_results_df = pd.DataFrame(recurrence_results)
print("\nSegment vs Recurrence Rate:")
print(recurrence_results_df.to_string(index=False))
recurrence_results_df.to_csv('segment_vs_recurrence.csv', index=False)

# Kruskal-Wallis test for number of affected segments vs treatment type
df['num_segments'] = df['segment'].str.split(',').str.len()
surgical = df[df['treatment_type'] == 'Surgical']['num_segments']
conservative = df[df['treatment_type'] == 'Conservative']['num_segments']
h_statistic, p_value = kruskal(surgical, conservative)

print("\nKruskal-Wallis Test for Number of Affected Segments vs Treatment Type:")
print(f"H-statistic: {h_statistic}")
print(f"p-value: {p_value}")

# Visualization: Number of affected segments vs Treatment Type
plt.figure(figsize=(10, 6))
sns.boxplot(x='treatment_type', y='num_segments', data=df)
plt.title('Number of Affected Segments vs Treatment Type')
plt.savefig('segments_vs_treatment_boxplot.png')
# plt.close()
plt.show()

# Kaplan-Meier analysis for recurrence
# Note: This is a simplified version as we don't have time-to-event data
kmf = KaplanMeierFitter()

plt.figure(figsize=(10, 6))
for treatment in df['treatment_type'].unique():
    mask = df['treatment_type'] == treatment
    kmf.fit(df[mask]['num_segments'], df[mask]['recidivă'], label=treatment)
    kmf.plot()

plt.title('Kaplan-Meier Estimate of Recurrence-Free Probability')
plt.xlabel('Number of Affected Segments')
plt.ylabel('Recurrence-Free Probability')
plt.savefig('kaplan_meier_recurrence.png')
# plt.close()
plt.show()

print("\nAnalysis complete. Results have been saved to CSV files and plots have been generated.")

Segment vs Treatment Decision:
Segment           Test Type  p-value Significant
     l3     Chi-square test 1.000000          No
     l4     Chi-square test 0.882704          No
     l5     Chi-square test 0.238514          No
            Chi-square test 1.000000          No
     s1     Chi-square test 0.928641          No
     t1 Fisher's exact test 0.477158          No
     t2 Fisher's exact test 0.437500          No
     t3 Fisher's exact test 0.437500          No
     t4 Fisher's exact test 1.000000          No
     t9 Fisher's exact test 0.305504          No
    t10 Fisher's exact test 0.305504          No
     c5 Fisher's exact test 1.000000          No
     c6 Fisher's exact test 0.437500          No
     c7 Fisher's exact test 1.000000          No
     t5 Fisher's exact test 0.573832          No
     t6 Fisher's exact test 0.305504          No
     l2     Chi-square test 0.902295          No
    t11 Fisher's exact test 1.000000          No
    t12 Fisher's exact test 1.000000  


Analysis complete. Results have been saved to CSV files and plots have been generated.
